# POS Tagging

#### Problem Statement: 
_The vanilla Viterbi algorithm we had written had resulted in ~87% accuracy. The approx. 13% loss of accuracy was majorly due to the fact that when the algorithm encountered an unknown word (i.e. not present in the training set, such as 'Twitter'), it assigned an incorrect tag arbitrarily. This is because, for unknown words, the emission probabilities for all candidate tags are 0, so the algorithm arbitrarily chooses (the first) tag._

_In this assignment, you need to modify the Viterbi algorithm to solve the problem of unknown words_

This assignment is done using 2 methods:

- Since the emission probabilities of the unknow words are "0", the state probability becomes "0" (_as a result of the product emission probability * transistion Probability_), to avoid that, for all the unknowns only the transtational probabilities are used.

- Seconldy using unigram tagger & regex along with veterbi

In [1]:
#import Libs
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random, pprint, time
from collections import Counter

#import train test split
from sklearn.model_selection import train_test_split


In [2]:
#read the data
data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
print('Total number of words in data are: ',len(data))

Total number of words in data are:  3914


In [4]:
data[:2]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')]]

In [5]:
#splliting the data into train & test set
random.seed(1234)
train_set, test_set = train_test_split(data, test_size=.05)

In [6]:
print('Shape of the train set:', len(train_set))
print('Shape of the test set:', len(test_set))

Shape of the train set: 3718
Shape of the test set: 196


In [7]:
#creating tag of words for both train & test data
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)


# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]
#print(test_run_base)
# list of untagged words
test_tagged_words = [tup[0] for tup in test_run_base]
#test_tagged_words

train_tagged_words = [tup for sent in train_set for tup in sent]

In [8]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Mr.',
 'Oxnard',
 'observed',
 'that',
 'the',
 'situation',
 'in',
 'Brazil',
 'is',
 'also']

In [9]:
# vocabulary
V = set(tokens)
print(len(V))

12081


In [10]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(len(T))
print(T)

12
{'VERB', 'DET', 'PRON', '.', 'PRT', 'ADV', 'X', 'NOUN', 'CONJ', 'ADP', 'ADJ', 'NUM'}


In [11]:
# Creating matrix for tags and words
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

## Baseline Veterbi implementation

In [12]:
#Computing emission probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    '''
Function to calculate p(w|t)
    '''
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list) # Denominator
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)#numerator
    return (count_w_given_tag, count_tag)


In [13]:
#Computing the transistion probabilities
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    '''
Function to calculate p(t2|t1)

    '''

    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [16]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [17]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
print('Time required for execution:',end - start)

Time required for execution: 823.018013715744


In [18]:
check = list([i for i, j in zip(tagged_seq, test_run_base) if i == j])
len(check)

4687

In [19]:
accuracy = len(check)/len(tagged_seq)
accuracy

0.922456209407597

In [20]:
#opening the valiation file 
with open('Test_sentences.txt') as f:
    validation_data = f.read()

#tokenization of Validatword_tokenizeenizeenize
validation_token = nltk.word_tokenize(validation_data)

In [21]:
#Testing the validation data
start = time.time()
tagged_seq_validation = Viterbi(validation_token)
end = time.time()
print('Time required for execution:',end - start)


Time required for execution: 30.368805170059204


In [22]:
tagged_seq_validation

[('Android', 'VERB'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'VERB'),
 ('.', '.'),
 ('Android', 'VERB'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'VERB'),
 ('worldwide', 'VERB'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'VERB'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'VERB'),
 ('.', '.'),
 ('Google', 'VERB'),
 ('and', 'CONJ'),
 ('Twitter', 'VERB'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'VERB'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'VERB'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'VERB'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'VERB'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 

### Observations from the Validation set, while using baseline Verterbi 
- _From the data it seems that the words such as Andriod, Google, Twitter are marked as DET instead of Noun, this behaviour changes with each execution!_
- _Some problem with tagging numbers, for example 2018, is tagged as DET_

In [23]:
#checking the incorrected tags in the test data to check the problems
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[("n't", 'ADV'), (('that', 'ADP'), ('that', 'ADV'))],
 [('Italy', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('is', 'VERB'), (('Angelo', 'VERB'), ('Angelo', 'NOUN'))],
 [('Angelo', 'NOUN'), (('Gaja', 'VERB'), ('Gaja', 'NOUN'))],
 [('Gaja', 'NOUN'), (('Barbaresco', 'VERB'), ('Barbaresco', 'NOUN'))],
 [(',', '.'), (('Piero', 'VERB'), ('Piero', 'NOUN'))],
 [('Piero', 'NOUN'), (('Antinori', 'VERB'), ('Antinori', 'NOUN'))],
 [('La', 'NOUN'), (('Solaia', 'VERB'), ('Solaia', 'NOUN'))],
 [('net', 'ADJ'), (('importer', 'VERB'), ('importer', 'NOUN'))],
 [('``', '.'), (('genuine', 'VERB'), ('genuine', 'ADJ'))],
 [('this', 'DET'), (('touchy', 'VERB'), ('touchy', 'ADJ'))],
 [('pay', 'VERB'), (('about', 'ADP'), ('about', 'ADV'))],
 [('$', '.'), (('77,000', 'VERB'), ('77,000', 'NUM'))],
 [('for', 'ADP'), (('free', 'ADJ'), ('free', 'ADV'))],
 [('$', '.'), (('14.75', 'VERB'), ('14.75', 'NUM'))],
 [('No', 'DET'), (('one', 'NUM'), ('one', 'NOUN'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'A

_Unknown words, which are not in training dataset are getting tagged incorrectly, to improve the efficency of the model, lets do some modification on the Veterbi algorithm._

## Modification #1

_In this method, its implied that unkown words only has the transition probabilities_

In [24]:
#modification #2
def viterbi_modification_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    tokens = list(set([pair[0] for pair in train_bag]))
    unknown_word = [word for word in words if word not in tokens]
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            if word in unknown_word:
                state_probability = transition_p
            else:
                state_probability = emission_p * transition_p
            
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [25]:
start = time.time()
tagged_seq_1 = viterbi_modification_1(test_tagged_words)
end = time.time()
print('Time required for execution:',end - start)

Time required for execution: 810.278270483017


In [26]:
check_1 = [i for i, j in zip(tagged_seq_1, test_run_base) if i == j]

In [27]:
accuracy_modification_1 = len(check_1)/len(tagged_seq_1)
accuracy_modification_1

0.940759692973824

In [28]:
#checking the incorrected tags in the test data to check the problems
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_1, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[("n't", 'ADV'), (('that', 'ADP'), ('that', 'ADV'))],
 [('Italy', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('is', 'VERB'), (('Angelo', 'X'), ('Angelo', 'NOUN'))],
 [('Angelo', 'NOUN'), (('Gaja', 'VERB'), ('Gaja', 'NOUN'))],
 [('Gaja', 'NOUN'), (('Barbaresco', 'X'), ('Barbaresco', 'NOUN'))],
 [('``', '.'), (('genuine', 'NOUN'), ('genuine', 'ADJ'))],
 [('this', 'DET'), (('touchy', 'NOUN'), ('touchy', 'ADJ'))],
 [('pay', 'VERB'), (('about', 'ADP'), ('about', 'ADV'))],
 [('$', '.'), (('77,000', 'NOUN'), ('77,000', 'NUM'))],
 [('for', 'ADP'), (('free', 'ADJ'), ('free', 'ADV'))],
 [('$', '.'), (('14.75', 'NOUN'), ('14.75', 'NUM'))],
 [('No', 'DET'), (('one', 'NUM'), ('one', 'NOUN'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('.', '.'), (('Arbitrage-related', 'NOUN'), ('Arbitrage-related', 'ADJ'))],
 [('of', 'ADP'), (('buy', 'VERB'), ('buy', 'NOUN

In [29]:
#runnnig the algorithm on the validation data

start = time.time()
tagged_seq_validation_1 = viterbi_modification_1(validation_token)
end = time.time()
print('Time required for execution:',end - start)

Time required for execution: 27.765124320983887


In [30]:
tagged_seq_validation_1

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'DET'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'DET'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('servic

### Observations from the Validation set

- The situation has improvied a bit, this is also seen by the accuracy (~93%).
- But still, some words like google, smartphones are tagged as determinents.
- Numbers still has issues

## Modification #2

_I this case, using a Unigram tagger along with a regex tagger to supplement the veterbi algorithm_

In [31]:
def viterbi_modification_2(words, train_bag = train_tagged_words):
    
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    patterns = [
    (r'.*ing$', 'VERB'),
    (r'.*ed$', 'VERB'),  
    (r'.*es$', 'VERB'),
    (r'.*ould$', 'VERB'), 
    (r'.*\'s$', 'PRT'),               
    (r'.*s$', 'NOUN'), 
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),
    (r'.*', 'NOUN')]

    regexp_tagger = nltk.RegexpTagger(patterns)

# lexicon backed up by the rule-based tagger
    lexicon_tagger = nltk.UnigramTagger(train_set, backoff=regexp_tagger)

    #lexicon_tagger.evaluate(incorrect_tag_word)
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = T[p.index(pmax)]
        #state_max = []
        if (pmax == 0):
            
            new_tag = lexicon_tagger.tag((word,state_max))
            
            state_max = new_tag[0][1]
            
        state.append(state_max)
    return list(zip(words, state))


In [32]:
start = time.time()
tagged_seq_2 = viterbi_modification_2(test_tagged_words)
end = time.time()
print('Time required for execution:',end - start)


Time required for execution: 798.3143727779388


In [33]:
check_2 = [i for i, j in zip(tagged_seq_2, test_run_base) if i == j]
print(len(check_2))
print(len(test_run_base))
print(len(tagged_seq_2))

4825
5081
5081


In [34]:
accuracy_modification_2 = len(check_2)/len(tagged_seq_2)
accuracy_modification_2

0.949616217280063

In [35]:
#checking the incorrected tags in the test data to check the problems
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq_2, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[("n't", 'ADV'), (('that', 'ADP'), ('that', 'ADV'))],
 [('Italy', 'NOUN'), (('there', 'DET'), ('there', 'ADV'))],
 [('``', '.'), (('genuine', 'NOUN'), ('genuine', 'ADJ'))],
 [('this', 'DET'), (('touchy', 'NOUN'), ('touchy', 'ADJ'))],
 [('pay', 'VERB'), (('about', 'ADP'), ('about', 'ADV'))],
 [('for', 'ADP'), (('free', 'ADJ'), ('free', 'ADV'))],
 [('No', 'DET'), (('one', 'NUM'), ('one', 'NOUN'))],
 [(',', '.'), (('though', 'ADP'), ('though', 'ADV'))],
 [('such', 'ADJ'), (('close', 'NOUN'), ('close', 'ADJ'))],
 [('acquiring', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('.', '.'), (('Arbitrage-related', 'VERB'), ('Arbitrage-related', 'ADJ'))],
 [('of', 'ADP'), (('buy', 'VERB'), ('buy', 'NOUN'))],
 [('the', 'DET'), (('close', 'ADJ'), ('close', 'NOUN'))],
 [('to', 'PRT'), (('augment', 'NOUN'), ('augment', 'VERB'))],
 [('buy', 'VERB'), (('more', 'ADV'), ('more', 'ADJ'))],
 [('company', 'NOUN'), (('that', 'ADP'), ('that', 'DET'))],
 [('that', 'DET'), (('*T*-146', 'NOUN'), ('*T*-146', 'X

In [36]:
#runnnig the algorithm on the validation data

start = time.time()
tagged_seq_validation_2 = viterbi_modification_2(validation_token)
end = time.time()
print('Time required for execution:',end - start)

Time required for execution: 29.496506690979004


In [37]:
tagged_seq_validation_2

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

### Observations from Method 2 
- The situation has improved quite a lot (~95%)
- The baseline Viterbe algorithm gives an accuracy of 91%, where as method 1 & method 2, increases this.
- The numbers are identified correctly.
- Android, Google, twitter etc has been **identified correctly by method 2** 

### Comaparison from Baseline Veterbi

- The baseline verterbi methods proves as a good starting point, but doesn't performs well on unknown items.
- The basic problem with unknown words is, the algorithm give "0" probability as the emission probabilities are not known.
- By assuming on transitional probabilies for the unknow words the sitution improves (93% accracy).
- Finally, the minor issues are resolved by the unigram & regrex Tagger & hence the model improves further (~95% accuracy)